This script loads pasted SMILES and inserts random smiles from the top-x of a dataset. The script creates A list of lists of lists where the first list contains the 30 initial startings lists of molecules for each run with 1 molecule nudged. The second list contains 30 initial starting lists with 2 molecules nudged and so on.

In [1]:
from MDRMF.dataset import Dataset
import random
import numpy as np

In [2]:
dataset = Dataset.load('../datasets/dataset_mqn_shuffled.pkl')
dataset.remove_duplicates()
dataset

<Dataset X.shape: (9892, 42), y.shape: (9892,), w.shape: (9892,), ids: ['NC(=O)CCN(Cc1ccccc1)C(=O)c1cn(C2CC[NH2+]CC2)nn1'
 'CN(C[C@H]1CC[N@@H+](C)C1)c1nc(-c2cccnc2)nc2ccccc12'
 'O[C@@H]1CCCC[C@H]1Nc1ccnc(-c2ccccc2)n1' ...
 'COc1cccc2c1CC[C@@]2(N)C(=O)NCc1ccn(C(C)C)n1'
 'Cc1nnc(Cn2nnc(C(=O)N3CCC(Cc4ccccc4)CC3)c2C)o1'
 'CCC(CC)n1nc(C)cc1C(=O)N[C@H]1C=C[C@H](CO)C1']>

In [3]:
# List of SMILESs used in previous experiments
SMILES = [
    [
        'O=C(Nc1ccc(Nc2ncccn2)cc1)c1cn[nH]c1-c1ccco1',
        'NC(=O)c1ccc(C(=O)N2CCC[C@H](Cn3ccnn3)C2)nc1',
        'COc1ccnc(NC[C@]23C[C@](NC(=O)[C@@H]4C[C@@H]4C)(C2)C(C)(C)O3)n1',
        'Cc1csc(N2CCN(C(=O)c3ccc(C(=O)NC4CC4)cc3)C[C@H]2C)n1',
        'CN1C(=O)CCc2cc(NC(=O)NC[C@@H](O)c3ccccc3)ccc21',
        'O=C(Nc1cccnc1)c1cccc(S(=O)(=O)N2CCCCC2)c1',
        'Cc1cc([N+](=O)[O-])c(C)c(S(=O)(=O)N(C)[C@@H]2C[N@H+]3CC[C@@H]2CC3)c1',
        'Cc1ccc(C(=O)N2CC[C@H](c3cc(O)[nH]n3)C2)cc1',
        'Cc1cccc([C@@H]2CCCN2C(=O)NCc2ccc(S(C)(=O)=O)cc2)n1',
        'Cc1ccc(C[C@@H](C)NC(=O)N2CCS(=O)(=O)CC2)s1'
    ],
    [
        'O=C([O-])c1cccc(CS(=O)(=O)N2CC[C@H](O)C2)c1',
        'O=C(CCc1cccc(Br)c1)N[C@H]1C[C@H](Cn2ccnc2)C[C@@H]1O',
        'Cc1ccccc1CNc1cc(C(N)=O)ccc1Cl',
        'COc1ccc(OC)c([C@@H]2CCCN2C(=O)c2ccnc(OC)n2)c1',
        'C=CCN(CC(=O)[O-])S(=O)(=O)c1ccc(OC)c(Cl)c1',
        'O=C(N[C@H]1C[C@@H](Cn2ccnc2)C[C@H]1O)c1ccc2ccccc2c1',
        'COC[C@@H]1CN(C(=O)c2ncoc2C2CC2)CCO1',
        'COCCN1C(=O)N[C@H]2CN(C(=O)c3cccc(C#N)c3)CC[C@@H]21',
        'CCc1nocc1CN(C)C(=O)c1ccc2[nH]c(=O)n(C)c2c1',
        'CC[NH+]1CCN([C@@H]2CCN(C(=O)Cc3cncs3)C2)CC1'
    ],
    [
        'CC(C)C[C@H]1C(=O)NCCN1C(=O)c1cc(O)nc(C(C)C)c1',
        'O=C(Cc1ccc(-n2cnnn2)cc1)Nc1cnn(C[C@@H]2CCCO2)c1',
        'Cc1nc(NCc2cccc(N3CCCCC3=O)c2)cc(-c2ccccn2)n1',
        'O=C(c1ccnn1C1CCCC1)N1CCN(c2cncc3ccccc23)CC1',
        'O=C(NCC1([C@@H](O)c2ccccc2)CC1)c1cnn2c(=O)cc[nH]c12',
        'N#Cc1ccc(C(=O)N2CCCC[C@H]2C(=O)[O-])cc1',
        'Cc1cc(C)n(CC(=O)NC[C@@H]2CC[C@@H](c3ncn[nH]3)O2)n1',
        'Cc1c(CC(=O)N2CC[C@@H](S(C)(=O)=O)C2)nnn1-c1ccccc1',
        'CC(C)CCOC1CCN(C(=O)c2ccc(C(=O)[O-])cn2)CC1',
        'CN(C)c1cccc(S(=O)(=O)N2CCC[C@@H](n3cc(C(=O)[O-])cn3)C2)c1'
    ],
    [
        'CCc1nn(C)cc1C(=O)N[C@@H]1c2ccccc2C[C@@H]1O',
        'COc1ccccc1CN(C[C@H]1CCCO1)C(=O)c1cccc2[nH]c(=O)oc12',
        'C=CCC1(C(=O)N2CC[C@@H](c3nc(C)no3)C2)CCOCC1',
        'CC[C@@H](CC#N)NC(=O)c1nnc2ccc(Br)cn12',
        'O=C(c1cccc(Cn2ccnn2)c1)N1C[C@H]2COC[C@@H]2C1',
        'C[C@H](Cc1nc(-c2ccccc2)no1)NC(=O)c1cccc(C#N)c1',
        'CSC[C@@H](C)C(=O)N1CCOC[C@H]1CC(=O)[O-]',
        'C[C@H]1CN(c2nnc(-c3ccccc3)n2C[C@@H]2CCCO2)CCS1',
        'COCC1(C)CCN(C(=O)Nc2ccc(-n3cccn3)nc2)CC1',
        'O=C(c1ncsc1Nc1ccccc1)N1C[C@@H](CO)[C@@H](CO)C1'
    ],
    [
        'Cc1cc(C)cc(-n2nc(C(=O)N3CCCC[C@@H]3c3cn[nH]c3)cc2C)c1',
        'CCc1cccc(C)c1NS(=O)(=O)c1cnc2n1CCC2',
        'O=C(Nc1cnn(-c2ncccc2F)c1)C1(F)CCC1',
        'CCn1c(=O)c(=O)[nH]c2cc(C(=O)NC[C@H]3CN(C)CCO3)ccc21',
        'CCC[C@H]1CCCC[C@H]1CNC(=O)N1CC(=O)N(C)C1',
        'O=C(NC[C@H]1CCC[N@H+](Cc2ncon2)C1)c1cccc(-n2cncn2)c1',
        'Cc1cccc2nc(N3CC[C@H](S(C)(=O)=O)C3)sc12',
        'CC(C)c1cccc(S(=O)(=O)NC(=O)CC[C@H]2Cc3ccccc3NC2=O)c1',
        'CCc1onc(C)c1NS(=O)(=O)c1ccc(OCC(=O)[O-])c(Cl)c1',
        'CC(=O)Nc1ccc(NC(=O)c2cccc3ncn(C)c23)cc1'
    ],
    [
        'NC(=O)c1cccc(Oc2ccc(NC(=O)[C@H]3C[C@@H]3c3ccc(F)cc3)cn2)c1',
        'C[C@H]([NH3+])[C@@H]1CCCN(c2ccc(-c3ccc(Cl)s3)nn2)C1',
        'CNC(=O)[C@@H]1CCC[C@H]1[NH2+][C@H]1[C@@H]2CCO[C@@H]2C12CCCC2',
        'CCc1cc(N2CC[C@@H](n3cc(C(=O)[O-])nn3)C2)c2ccccc2n1',
        'C[C@@]1(c2cnccn2)NC(=O)N(Cc2noc3c2CCCC3)C1=O',
        'CCc1onc(C)c1NC(=O)NCC1CCc2ccccc2CC1',
        'Cc1ccc(F)cc1C(=O)NC(C)(C)c1cn(C)nn1',
        'CCOc1c(Cl)cc(CNc2ccc3nc(C)nn3n2)cc1OC',
        'O=C(C[C@@H]1CCc2ccccc2C1)N1CCC(c2nnc3n2CC[NH2+]C3)CC1',
        'Cc1ccsc1S(=O)(=O)NC(=O)C(F)(F)C1CCOCC1'
    ],
    [
        'O=C(COc1ccc(F)cc1)N[C@H]1CCC(=O)N[C@H]1c1ccccc1',
        'CCc1ccsc1C(=O)N1CCC(N2CCNC2=O)CC1',
        'CCS(=O)(=O)N1CCN(c2nnc(-c3cccn3C)n2C)CC1',
        'COCC1(C)CCN(C(=O)Nc2cnn(-c3ncccn3)c2)CC1',
        'O=C(c1ccccc1)N1CCN(C(=O)C2CCOCC2)CC1',
        'O=C(NCc1ccc(Br)s1)NC[C@H]1CCCO1',
        'O=C(NCc1ccc(-n2ccnc2)nc1)Nc1ccc(CN2CCCC2=O)cc1',
        'O=C([O-])C[C@H](NC(=O)c1ccc2cc[nH]c2c1)c1cccs1',
        'Cc1nc(CN(C)C(=O)C[C@H]2CCCC[C@H]2O)no1',
        'CS(=O)(=O)[C@@H]1CCN(C(=O)c2cccnc2N2CCCC2)C1'
    ],
    [
        'CCn1nc(C)c(C[N@H+]2CCc3[nH]cnc3[C@@H]2c2ccc(F)cc2)n1',
        'COC(=O)c1csc(NC(=O)[C@@]2(C)CCN(C(C)=O)C2)c1',
        'Cc1nc(N2CCOCC2)ccc1NC(=O)N1CC(CO)C1',
        'COc1ccccc1[C@@H](CNC(=O)N1CC[C@](F)(CO)C1)[NH+]1CCCC1',
        'CCOc1ccccc1C(=O)N1CCC[C@@H]1C(=O)Nc1cnn(CCOC)c1',
        'Cc1cccc(CN(C)C(=O)c2cnn3ccc(C)cc23)c1',
        'CC(C)N(C)C(=O)C1CCN(C(=O)CCc2ccc(C(=O)[O-])cc2)CC1',
        'CC1CCN(C(=O)C2CCN(C(=O)Nc3cccc(-c4ccc5c(c4)CCO5)c3)CC2)CC1',
        'COc1cc(CC(=O)N2CCOC[C@H]2c2c(C)n[nH]c2C)ccc1C',
        'CCn1cc(Cl)cc1C(=O)N[C@@H](C(=O)[O-])C(C)C'
    ],
    [
        'CCn1nc(C)c(Cn2cnc3c(sc4ncccc43)c2=O)n1',
        'O=C(Nc1cnn(C[C@H]2CCCO2)c1)N[C@H](CN1CCCC1=O)c1ccccc1',
        'Cc1c(Br)cc(F)cc1C(=O)N(CC(N)=O)CC(C)C',
        'CS(=O)(=O)NCc1cn(-c2cccc(C(F)(F)F)c2)nn1',
        'COc1c(C)cc(S(=O)(=O)N2CCN(C(=O)C3CC3)C2)cc1C',
        'Cc1nc(C(=O)N(C)C)nc(N2CC[C@H](Nc3ncc(C#N)cc3F)C2)c1C',
        'O=C(NCC(=O)N1CCc2ccccc2C1)[C@@H]1C[C@H](O)CN1C(=O)OCc1ccccc1',
        'CC(C)(C)c1cc(Cn2cnc3sccc3c2=O)no1',
        'COc1cccc(C(=O)NCC(=O)N[C@@H]2CCC(=O)N[C@H]2[C@@H]2CCCO2)c1',
        'O=C(Cn1cnc2scc(-c3ccc(Br)cc3)c2c1=O)N1CCCC1'
    ],
    [
        'CC(=O)N1c2ccc(S(=O)(=O)N3CC4(CCC4)c4c(F)cccc43)cc2C[C@@H]1C',
        'Cn1ccnc1[C@H]1CCN(C(=O)Cc2n[nH]c(=O)c3ccccc23)C1',
        'COC[C@H](Nc1ccc2nc(C3CC3)oc2c1)C1CCOCC1',
        'Cc1cc(S(N)(=O)=O)cc(C(=O)NC[C@@H]2CCCO2)c1C',
        'CC[C@@H](CO)NC(=O)Nc1ccc(S(=O)(=O)NC(C)(C)C)cn1',
        'CC(C)c1cccc(S(=O)(=O)NC(=O)CC[C@H]2Cc3ccccc3NC2=O)c1',
        'COc1ccc(-c2nc(CCNC(=O)c3sc(C)nc3C)co2)cc1',
        'CCc1csc(CC(=O)N[C@@H](CN2CCOCC2)c2ccc(C)cc2)n1',
        'O=C(NCCc1ccc2cn[nH]c2c1)N1CCCC1',
        'Cc1csc([C@H]2CCCN2C(=O)c2ccc3cc[nH]c3n2)n1'
    ],
    [
        'O=C(CN1CCc2ccccc2C1=O)NCc1ccccc1',
        'COC(=O)[C@@H]1CCc2nnc(C(=O)N3CCCC3)n2C1',
        'O=C(CNC(=O)[C@@H]1CCC[NH2+]1)Nc1ccc2c[nH]nc2c1',
        'COc1cc2c(cc1F)CC[C@@H]2NC(=O)Cn1c(C)nc2ccccc2c1=O',
        'C[C@H]1C[C@@H]1C(=O)Nc1n[nH]c2nc(C(F)(F)F)ccc12',
        'Cn1ccnc1[C@]1(CNC(=O)c2ccc(-c3nnn[nH]3)cn2)NC(=O)NC1=O',
        'Cc1n[nH]c(C)c1CC(=O)N1CCO[C@H](c2ccccc2)C1',
        'O=C(CC[C@@H]1CCCOC1)NS(=O)(=O)Cc1cccc(F)c1',
        'Cc1nnc(-c2ccc(NC(=O)N3CCCC3)cc2)o1',
        'CC(C)C(=O)Nc1cc(C(=O)N2CCC(O)(c3ccccc3)CC2)ccn1'
    ],
    [
        'Cc1ccccc1Cc1noc(C2CCN(C(=O)N(C)C)CC2)n1',
        'CC(C)Cc1cc(N2CCC(Cc3nncn3C)CC2)ncn1',
        'CCNC(=O)N1CCC[C@@H]1c1nc(-c2ccccc2)c(C(C)C)o1',
        'COc1cc(C)c([C@H](C)NC(=O)N2CCC2)cc1OC',
        'CC(C)N(C)C(=O)C1CCN(C(=O)CCc2ccc(C(=O)[O-])cc2)CC1',
        'CC(C)Cn1ncc(C(=O)NCC2(O)CCCCCC2)c1C(F)F',
        'Cc1nccn1-c1ncccc1CNC(=O)C1(c2ccccc2Cl)CCOCC1',
        'C[NH+]1CCC(CNC(=O)CNC(=O)c2ccc3ccccc3c2)CC1',
        'CCn1cc(C(=O)NCC2([C@H](O)c3ccccc3)CC2)c2ccccc21',
        'Cc1ccc(S(=O)(=O)N[C@@H]2C=C[C@H](C(=O)[O-])C2)cc1C'
    ],
    [
        'C[C@@H](c1ccc(F)c(F)c1)N(C)C(=O)c1cn(Cc2cccc(F)c2)nn1',
        'O=C(NCc1ncccn1)c1ccc2c(c1)C[C@H](c1ccccc1)OC2=O',
        'Cc1cc(S(=O)(=O)NOCC(=O)[O-])ccc1OC(C)C',
        'CC1CCN(S(=O)(=O)N[C@H]2CCN(c3ccccc3)C2=O)CC1',
        'CCc1nc([C@@H](C)NC(=O)N[C@@H]2CCCc3cn[nH]c32)n[nH]1',
        'Cn1ccc2ccc(NC(=O)N3CC[C@H](C(C)(C)C)C3)cc2c1=O',
        'Cc1nc(C2CC2)nc(C)c1C(=O)NCC1CCC1',
        'Cc1ccc(NC(=O)N2CCN(C(=O)c3ccoc3C(F)(F)F)CC2)cc1',
        'CC1(c2nc(-c3ccnc(N4CCOCC4)c3)no2)CC1',
        'C[C@]1(C(=O)[O-])CCC[N@@H+]1CC(=O)Nc1ccc(F)cc1Cl'
    ],
    [
        'C[C@H](NC(=O)c1ccc(Nc2ncccn2)cc1)[C@H](O)c1ccccc1',
        'O=C(NCc1ccc(Oc2cccnc2)c(F)c1)N1CC[C@H](C2CCOCC2)C1',
        'CCCNC(=O)C[NH+]1CCC(Nc2cc(C(F)(F)F)nc3ncnn23)CC1',
        'Cc1ccc(Cl)c(S(=O)(=O)NC(=O)C(C)C)c1Cl',
        'CS(=O)(=O)c1ccc(NC(=O)NC2CCOCC2)cn1',
        'O=C(Nc1cccc2ccccc12)N[C@H]1CC[NH2+]C1',
        'CCOc1ccc(CNC(=O)N2CCC[C@H](C(=O)NC)C2)cc1F',
        'O=C(c1c[nH]nc1-c1cc2ccccc2o1)N1CCO[C@]2(CCCOC2)C1',
        'C=C(C)CCNC(=O)[C@H]1CC(c2ccccc2)=[NH+]O1',
        'O=C(NCc1cccc(Cn2cnc3ccccc32)c1)N[C@H]1CC[N@@H+]2CCC[C@@H]2C1'
    ],
    [
        'CC(C)CCn1ncc(NC(=O)[C@@H]2CC23CCOCC3)c1C(C)C',
        'CCc1cccc(C[NH2+][C@H]2CCC(=O)N[C@@H]2c2cnn(C)c2)c1',
        'C[C@H](COc1ccc(F)cc1F)NC(=O)N1CCC2(CCO2)CC1',
        'O[C@H]1CC[C@@H](Nc2ccc(Br)cn2)CC1',
        'CS(=O)(=O)c1ccccc1N1CCC[C@H]1c1cccnc1',
        'Cc1nn(C)c2nc(-c3cccs3)cc(C(=O)NCc3ccccc3Cn3cncn3)c12',
        'CNC(=O)C1(c2nc(-c3ccc(N4CCCC4=O)cc3)no2)CCC1',
        'Cc1cc(N2CCC(CO)CC2)nc(-c2cccnc2)n1',
        'COc1ccccc1OCCN(C)C(=O)c1cc(=O)[nH]c2ccccc12',
        'CNC(=O)[C@H]1CCCCN1C(=O)Cc1ccc2c(c1)CCO2'
    ],
    [
        'Cc1ccc(S(=O)(=O)Cc2nnc(C)n2C)cc1C',
        'COCC(=O)N1CCN(C(=O)c2cccc(NCc3nc(C)cs3)c2)CC1',
        'O=C(c1ccc(-c2ccc(F)cc2)s1)N1CCN(c2cccnn2)CC1',
        'Cc1ccc2c(c1)C(=O)N([C@H](C)C(=O)Nc1cccnc1)C2=O',
        'CCS(=O)(=O)N1CCN(C(=O)C2(c3ccccc3)CC2)CC1',
        'Cc1ccc(CCC(=O)N[C@@H]2CCCCNC2=O)c(C)c1',
        'C[C@H]1C[N@@H+](C2CC2)C[C@@H]1NC(=O)N1Cc2ccc(Br)cc2C1',
        'Cc1cccc(F)c1CNC(=O)[C@H]1CC(=O)N(C2CCCC2)C1',
        'CC1(C)CCC(CNC(=O)Cc2nc3ncccn3n2)CC1',
        'CN(CCCNc1cccc2c1NC(=O)CO2)c1ccccc1'
    ],
    [
        'Cc1cc(COCC(=O)N(C)c2ccc(Br)cn2)no1',
        'COc1cc(NC(=O)CN2CCCCCC2=O)ccc1C(=O)[O-]',
        'Cc1nocc1NC(=O)NC[C@](C)(O)c1ccc(F)cc1',
        'CC(C)(C)c1cc(Cn2cnc3sccc3c2=O)no1',
        'COC(=O)c1c(NC(=O)NCCCC(C)C)nc2n1CCOC2',
        'Cc1cc(-c2noc(CCC(=O)N3CCC[C@@H]3C[C@H](O)c3ccc(F)cc3)n2)ccc1F',
        'Cn1cc(NC(=O)N2CCOC3(CCCC3)C2)c(-c2ccccc2)n1',
        'Cc1cc(C)c(S(=O)(=O)N2CCC=C(c3cnn(C)c3)C2)s1',
        'C[C@@](O)(CCc1ccccc1)C(=O)N1CCCc2n[nH]cc21',
        'O=C([O-])c1cc(F)ccc1NS(=O)(=O)c1c[nH]c2ncccc12'
    ],
    [
        'CCc1nc(CNc2cccc3c2C(=O)NC3)no1',
        'Cc1ccn2c(NC(=O)c3cc(F)ccc3Cl)nnc2c1',
        'N#Cc1cc(Br)cc(S(=O)(=O)NC(=O)Cc2ccc(F)cn2)c1',
        'O=C(CCc1nc2cccnc2[nH]1)N(Cc1ccccc1)C[C@@H](O)c1ccccc1',
        'CCOC(=O)Cn1cnc2cc(Cl)ccc2c1=O',
        'C[C@H]([NH2+]Cc1ccc2cn[nH]c2c1)c1ccccc1F',
        'C[C@H](Cc1cccc(Cl)c1)C(=O)N1CCCN(C(N)=O)CC1',
        'O=C(Nc1ccc2c(c1)COC2)c1coc(C2CCOCC2)n1',
        'COc1cc(C)c(C(=O)N2CCC[C@H]2c2nncn2C)cc1C(=O)[O-]',
        'C[C@@H](NC(=O)N1CCc2nccnc2C1)c1cccc(Cl)c1'
    ],
    [
        'O=C(COc1ccccc1F)N1CCCCC[C@@H]1c1ncon1',
        'CCc1cccc2c1N(C(=O)c1cc(OC)no1)CC2',
        'Cc1ccc(CNC(=O)CN2CCCC[C@@H](O)C2=O)cc1F',
        'O=C1[C@@H]([N@H+]2CCC[C@@H](Cn3ccnn3)C2)CCN1C1CC1',
        'CCc1c(NCc2ncnn2-c2ccccc2)cnn1-c1ccccc1C',
        'COc1cc2c(cc1F)CC[C@@H]2NC(=O)Cn1c(C)nc2ccccc2c1=O',
        'Cc1c(CCC(=O)N2CCN(C(=O)c3ccccc3)CC2)cnn1C',
        'CC1(C)OCC[C@H]1NC(=O)c1ccc(C(=O)[O-])cc1',
        'CC[C@H]1OCC[C@H]1NC(=O)Nc1nnn(C(C)C)n1',
        'Cc1nn(C)cc1[C@@H](C)NS(=O)(=O)N1CCOCC1'
    ],
    [
        'CCCC(=O)N1CCC[C@H]1C(=O)N[C@@H]1COc2ccccc2C1',
        'COc1ccc(Cl)cc1C(=O)N[C@H]1CCC(=O)NC1=O',
        'CC(C)S(=O)(=O)CC(=O)Nc1c2c(nn1C)CCC2',
        'O=C1CCCN1Cc1ccc(C(=O)N2CCC[C@@H](N3CCCS3(=O)=O)C2)cc1',
        'COc1cccc(CN(C)C(=O)c2ccc(C(=O)[O-])cn2)c1',
        'Cn1nc2c(c1NC(=O)[C@H]1CCCN(C(=O)Cc3ccccc3)C1)CCC2',
        'Cc1ccc2c(c1)C(=O)N([C@H](C)C(=O)Nc1cccnc1)C2=O',
        'Cc1cc(C)c(NC(=O)[C@H]2C[C@@H]2c2ccccc2C)c(C)n1',
        'CC(C)c1ncncc1C(=O)Nc1cccc(-c2nncn2C)c1',
        'CC1=[NH+]NC(=O)[C@@H]1CCNC(=O)c1nn(C)c2c1C[C@H](C)O[C@@H]2C'
    ],
    [
        'Cc1c[nH]cc1C(=O)N1CCC(c2ccc(S(N)(=O)=O)cc2)CC1',
        'COc1cc(CCO)ccc1NC(=O)[C@H]1CC1(C)C',
        'Cc1nnc2n1C[C@H](C(=O)NCC1(c3ccc(F)cc3Cl)CC1)CC2',
        'Cc1c(C2CC2)n[nH]c1NC(=O)c1cc(C#N)cc(N(C)C)c1',
        'Cn1ccnc1[C@@]1(CNC(=O)c2ccc(O[C@H]3CC3(F)F)cc2)NC(=O)NC1=O',
        'CN(Cc1ccc(F)cc1Cl)C(=O)c1cnns1',
        'COc1cc(OC)cc(C2(F)CCN(C(=O)c3ccc(=O)n(-c4cnn(C)c4)c3)CC2)c1',
        'CC(C)CC(=O)N1CCC(NC(=O)NC2CCOCC2)CC1',
        '[NH3+][C@H](CC(=O)NCC(=O)NC1CC1)Cc1cc(F)c(F)cc1F',
        'CN(CC1CC1)C(=O)Cn1ccc(-c2ccccc2)nc1=O'
    ],
    [
        'CCOc1ccc(NC(=O)N[C@H]2CCc3[nH]ncc3C2)cc1OC',
        'O=C([O-])CCC1CCN(C(=O)CCc2cccs2)CC1',
        'O=c1c2ccc(Cl)cc2ncn1C[C@@]1(O)CCCOC1',
        'CCc1sc(C(=O)NCc2nc(C)c(C(=O)[O-])s2)cc1C',
        'O=C(NC[C@@H]1CC[N@H+]1C1CCOCC1)c1cc(F)c2ncccc2c1',
        'COc1ccccc1NC(=O)Cc1noc([C@@H]2Cc3ccccc3N2C(C)=O)n1',
        'Cc1cccc(C(=O)N2CCOC[C@H]2CC(=O)[O-])c1Cl',
        'CC(C)(C)c1cnc(NC(=O)N[C@H]2C[C@@H](O)C2(C)C)s1',
        'Cc1noc([C@@H]2C[C@H](CNc3cnccn3)[C@@H](O)C2)n1',
        'Cc1cnc(C(=O)Nc2cccc(-c3nc(C)c(C)[nH]3)c2)cn1'
    ],
    [
        'Cc1ccc(S(=O)(=O)NC(=O)c2ccco2)cc1C',
        'O=C(Nc1ccccc1N1CCCC1)N[C@H]1[C@H]2CCO[C@H]2C12CCC2',
        'O=C1CN(C(=O)C2CCN(c3ccc4nncn4n3)CC2)CCN1',
        'OCCn1cc(Nc2ccnc(-c3ccccc3)n2)cn1',
        'COc1cc(C)cc(CNC(=O)[C@@H]2Cc3ccccc3N2C(C)=O)c1',
        'CN(C(=O)CS(=O)(=O)c1cc(F)ccc1F)C1CC1',
        'C[N@H+]1CCn2nc(NC(=O)NCc3ccccc3C(F)(F)F)cc2C1',
        'O=C(NC[C@H]1Cc2ccccc2C[NH2+]1)N1CCC2(CC1)CC2(F)F',
        '[NH3+][C@H]1CCC[C@@H](C(=O)Nc2ccc3nc(N4CCOCC4)sc3c2)C1',
        'Cc1nc(C(=O)N2CC=CCC23CCOCC3)nc(N2CC[C@@H]([NH3+])C2)c1C'
    ],
    [
        'CO[C@H]1C[C@H](COc2cccnc2)[N@@H+](Cc2ccnn2C)C1',
        'Cc1cc(NC(=O)c2cccc(F)c2)sc1C(=O)N1CC(C2CCOCC2)C1',
        'COc1ccccc1CC(=O)N1CCC(CC(=O)[O-])CC1',
        'COC(=O)c1cc(C)nc(N2CC[C@H](c3ccccc3)C2)n1',
        'O=C(CC[C@@H]1CC[NH2+]C1)Nc1cc(-c2cc(F)cc(F)c2)[nH]n1',
        'Cc1n[nH]c(C)c1CC(=O)N[C@H]1CC(C)(C)Oc2ccccc21',
        'COC(=O)Nc1ccc(NC(=O)[C@@H]2CCCN(c3cnccn3)C2)cc1',
        'Cc1ccc(CNc2nccc(C(C)(C)C)n2)c(N2CCCC2)n1',
        'Cc1cc(C(=O)N2CCC(CCC(=O)[O-])CC2)ccn1',
        'CN(c1ccccc1)S(=O)(=O)c1csc(C(=O)[O-])c1'
    ],
    [
        'O=C([O-])C1=[NH+]N(c2ccccc2Cl)C(=O)CC1',
        'Cc1cc(C(=O)[O-])ccc1S(=O)(=O)N[C@H](C)c1c(C)noc1C',
        'O=C(NCc1ccn(C2CCCC2)n1)c1cncnc1',
        'Cc1[nH]nc(C(=O)N2CCC(Oc3ccccc3Cl)CC2)c1C',
        'COc1cc(C)c([C@H](C)NC(=O)CN2CCOC3(CCC3)C2)cc1OC',
        'C[C@H](NC(=O)c1ccc(-n2cccn2)cc1)[C@@H](C)C(=O)[O-]',
        'O=C1NCc2nccc(N3CCC(c4ccccc4)CC3)c21',
        'CN(C(=O)N1CCC(c2nc(-c3ccccc3)n[nH]2)CC1)C1CCOCC1',
        'CC(C)n1cccc1C(=O)N[C@@H]1CC(=O)N(C(C)(C)C)C1',
        'O=C(N[C@H]1CCCN(c2ccccc2F)C1)c1cnn2cccnc12'
    ],
    [
        'CN(CC1(C)COC1)C(=O)Nc1cccc2nc(C(F)F)sc12',
        'CN(C)C(=O)N1CCN(C(=O)c2ncnc3ccsc23)CC1',
        'C[C@H](O)C(=O)N1CCN(C(=O)c2cn(Cc3cccs3)nn2)CC1',
        'CO[C@H]1C[C@@H](N(C)S(=O)(=O)c2ccc(OCC(=O)[O-])c(Cl)c2)C1',
        'O=C(C[C@@H]1CCCC(=O)N1)NCc1nnc(C2CC2)n1C1CC1',
        '[NH3+]Cc1nnc(N2CCCCC2)o1',
        'COCCn1cc(NS(=O)(=O)c2cnc3c(c2)c(C)nn3C)cn1',
        'FC(F)(F)c1cc(-c2cncc(-c3nnn[nH]3)c2)ccc1Cl',
        'CCc1nc([C@@H]2CCCN2C(=O)Nc2ccncc2)no1',
        'O=C([O-])c1coc(C[N@@H+]2CC=C(c3ccc(F)cc3)CC2)n1'
    ],
    [
        'COc1cc(F)cc(CNC(=O)N2CC[C@@]3(C2)C(=O)Nc2ccccc23)c1',
        'CO[C@H]1C[C@@H](N(C)C(=O)NC[C@@H]2CCCOC2)C1',
        'CC[C@@H](NC(=O)c1ccc(C)cc1C)c1c(C)noc1C',
        'CC1(C)C[C@@H](NC(=O)Nc2nnc(C(C)(C)C)s2)CCO1',
        'Cc1ccc(CC(=O)N2CCC(NC(=O)c3ccccc3)CC2)cn1',
        'O=C([O-])[C@@H]1CCN(S(=O)(=O)c2cccc(Cl)c2)CC1(F)F',
        'Cc1cccc(-c2nnc([C@H](C)[N@H+](C)CC3CCOCC3)o2)c1',
        'Cc1c(N2C(=O)[C@H]3C[C@H](O)CN3C2=O)nnn1-c1ccc(F)cc1',
        'CCc1ccc(C(=O)N2CC[N@@H+](C)C3(CCOCC3)CC2)o1',
        'CS(=O)(=O)N1CC[C@H](CNc2ncnc3ccccc23)C1'
    ],
    [
        'C[C@@H]1CCCN1C(=O)c1ccc(-c2nnn[nH]2)cc1',
        'O=S(=O)(NCc1cccnn1)c1ccccc1Br',
        'Cn1cc(-c2cncc(C(=O)N[C@H]3CC[C@H](O)CC3)c2)cn1',
        'CCOc1cn(-c2ccccc2)nc1C(=O)N1CCN(c2ncccn2)CC1',
        'Fc1ccc(OCc2noc([C@H]3C[C@]34CC[NH2+]C4)n2)cc1',
        'Cc1nn(Cc2ncc(-c3ccccc3)o2)c(C)c1[C@@H]1COCCN1C',
        'O=C(NCC1([C@H](O)c2ccccc2)CC1)c1cc(Cl)cc2cccnc12',
        'CCN(CC)C(=O)CCCNC(=O)Cc1cc2ccccc2c(=O)[nH]1',
        'Cc1c(F)cccc1S(=O)(=O)NCCc1cnoc1',
        'Cn1cc(S(=O)(=O)N[C@@H]2CCc3c(Cl)cccc32)cn1'
    ],
    [
        'NC(=O)c1cccc(Oc2ccc(NC(=O)[C@H]3C[C@@H]3c3ccc(F)cc3)cn2)c1',
        'Cc1[nH]c2ccccc2c1NC(=O)CCC(C)(C)C',
        'COc1ccc(CNC(=O)c2cnn(C(C)C)c2C)cc1',
        'O=C(NCc1ccccc1)N[C@@H]1CCCC[C@@H]1CO',
        'CC1CN(C(=O)c2ccnc(N3CC[C@H](C)C3)c2)C1',
        'CC[C@@H]1C[N@H+](C)CCCN1S(=O)(=O)c1nc(C)c(C)s1',
        'COc1ccc(C[NH2+]Cc2nnc3n2CCOCC3)c(Cl)c1',
        'O=C1Nc2ccccc2O[C@H]1CC(=O)N1CCC(c2c[nH]c3ncccc23)CC1',
        'COc1nc(C)cc(N(C)CC2(O)CCCC2)n1',
        'O=C(OCc1ccccc1)N1CCC[C@](O)(CN2CCOCC2)C1'
    ],
    [
        'O=C(c1cccnc1C(F)(F)F)N1CCS(=O)(=O)CC1',
        'CCc1ncnc(N2CCN(C(=O)c3ccco3)CC2)c1F',
        'Cc1ccncc1NC(=O)CC(C)(C)c1ccccc1',
        'CCC[C@H]1CCCC[C@H]1CNC(=O)N1CC(=O)N(C)C1',
        'C[C@H](NC(=O)c1c[nH]nc1-c1ccccc1F)c1ccc(NC(=O)c2ccncc2)cc1',
        'Cc1nc(NC(=O)[C@@H]2CCC[C@H](C(=O)[O-])C2)n[nH]1',
        'CN1CC[C@@H](NC(=O)Nc2cnn(-c3ccccc3Br)c2)C1=O',
        'O=C([O-])[C@]1(O)CCCN(S(=O)(=O)c2ccccc2Cl)C1',
        'CCOc1ccccc1CNC(=O)[C@@H]1C[C@@H](C)[C@@H](C(=O)[O-])O1',
        'Cc1nnc(-c2cccc(C(=O)N3CC(O)C3)c2)o1'
    ]
]    

In [4]:
def unique_random_numbers(n):
    rands = []
    i = 0
    while i < n:
        rand = random.randint(0,9)
        if rand in rands:
            continue
        else:
            rands.append(rand)
            i += 1

    return rands

unique_random_numbers(5)

[7, 5, 2, 9, 3]

In [5]:
best = dataset.get_top_or_bottom(100) # best mols.
dataset_best_sorted = Dataset.load('../datasets/dataset_mqn_shuffled.pkl')

new_SMILES = []

for i in range(5):
    temp_new_SMILES = []
    for j in range(30):
        best_samples = best.get_samples(i+1).ids.tolist()
        rands = unique_random_numbers(i+1)
        temp_SMILES = SMILES[j].copy()

        for k, rand in enumerate(rands):
            temp_SMILES[rand] = best_samples[k]
            
        temp_new_SMILES.append(temp_SMILES)
    new_SMILES.append(temp_new_SMILES)
    
dataset.get_points_from_ids(new_SMILES[4][0]).y

array([ -8.00715, -11.2869 ,  -8.86227, -11.1015 ,  -8.94156, -11.1258 ,
       -10.8185 ,  -8.78012, -10.963  ,  -5.67827])

In [6]:
dataset_best_sorted = Dataset.load('../datasets/dataset_mqn_shuffled.pkl')
dataset_best_sorted.sort_by_y()
middle_good_dataset = dataset_best_sorted.get_range(101, 500)

new_SMILES_middle_good = []

for i in range(5):
    temp_new_SMILES = []
    for j in range(30): 
        best_samples = middle_good_dataset.get_samples(i+1).ids.tolist()
        rands = unique_random_numbers(i+1)
        temp_SMILES = SMILES[j].copy()

        for k, rand in enumerate(rands):
            temp_SMILES[rand] = best_samples[k]
            
        temp_new_SMILES.append(temp_SMILES)
    new_SMILES_middle_good.append(temp_new_SMILES)
    
dataset.get_points_from_ids(new_SMILES_middle_good[4][0]).y

array([ -8.00715, -10.6877 ,  -8.86227, -10.1003 ,  -9.19134,  -9.70938,
       -10.1529 , -10.3027 ,  -8.78012, -10.4735 ])

In [7]:
new_SMILES

[[['O=C(Nc1ccc(Nc2ncccn2)cc1)c1cn[nH]c1-c1ccco1',
   'NC(=O)c1ccc(C(=O)N2CCC[C@H](Cn3ccnn3)C2)nc1',
   'COc1ccnc(NC[C@]23C[C@](NC(=O)[C@@H]4C[C@@H]4C)(C2)C(C)(C)O3)n1',
   'Cc1csc(N2CCN(C(=O)c3ccc(C(=O)NC4CC4)cc3)C[C@H]2C)n1',
   'CN1C(=O)CCc2cc(NC(=O)NC[C@@H](O)c3ccccc3)ccc21',
   'O=C(Nc1cccnc1)c1cccc(S(=O)(=O)N2CCCCC2)c1',
   'Cc1cc([N+](=O)[O-])c(C)c(S(=O)(=O)N(C)[C@@H]2C[N@H+]3CC[C@@H]2CC3)c1',
   'Cc1ccc(C(=O)N2CC[C@H](c3cc(O)[nH]n3)C2)cc1',
   'NS(=O)(=O)c1cccc(CNC(=O)NCc2cn(Cc3ccccc3)nc2-c2ccccc2)c1',
   'Cc1ccc(C[C@@H](C)NC(=O)N2CCS(=O)(=O)CC2)s1'],
  ['O=C([O-])c1cccc(CS(=O)(=O)N2CC[C@H](O)C2)c1',
   'O=C(CCc1cccc(Br)c1)N[C@H]1C[C@H](Cn2ccnc2)C[C@@H]1O',
   'Cc1ccccc1CNc1cc(C(N)=O)ccc1Cl',
   'COc1ccc(OC)c([C@@H]2CCCN2C(=O)c2ccnc(OC)n2)c1',
   'C=CCN(CC(=O)[O-])S(=O)(=O)c1ccc(OC)c(Cl)c1',
   'O=C(N[C@H]1C[C@@H](Cn2ccnc2)C[C@H]1O)c1ccc2ccccc2c1',
   'COC[C@@H]1CN(C(=O)c2ncoc2C2CC2)CCO1',
   'COCCN1C(=O)N[C@H]2CN(C(=O)c3cccc(C#N)c3)CC[C@@H]21',
   'C[C@@H]1Cc2ccccc2N1C1CCN(C(=O

In [8]:
len(new_SMILES)

5

In [9]:
# Tests list for number of unique molecules compared the list without enriched molecules.

count = 0
# Go through every enriched list of lists.
for nudged_lists in new_SMILES:
    
    # go through the 30 enriched lists. To compare them with the normal list we tuple them together with the non-enriched lists.
    ok_count = 0
    for unique_list, normal_list in zip(nudged_lists, SMILES):
        
        # if there is a unique molecule in a list add +1 to `unique_count`
        unique_count = 0
        for i in normal_list:
            if i in unique_list:
                pass
            else:
                unique_count += 1
        
        # a list have the correct number of unique molecules if it corresponds to count + 1.
        # That is, the first list of lists should have 1 unique aka count(0) + 1
        # the second list should have 2 unique and so on.
        if unique_count == (count + 1):
            ok_count += 1

    # For all list of lists to have the correct number of uniques, ok_count will equal 30.
    if ok_count == 30:
        print(f"All list {count+1} have {unique_count} unique(s)")
                
    count += 1

All list 1 have 1 unique(s)
All list 2 have 2 unique(s)
All list 3 have 3 unique(s)
All list 4 have 4 unique(s)
All list 5 have 5 unique(s)


In [10]:
# Test that each list contains 10 items.
count_nudged = 0
for nudged_lists in new_SMILES:
    
    count_unique = 0
    for unique_list in nudged_lists:
        if len(unique_list) != 10:
            print(f"Problem with nudge list {count_nudged} and unique list {count_unique}")
        count_unique += 1
    
    count_nudged += 1
print("Finished counting.")

Finished counting.


In [11]:
# Test that every item in every list is unique

missing_smiles = [smile for smile in temp_SMILES if smile not in dataset.ids]

for nudged_lists in new_SMILES:
    for unique_list in nudged_lists:
        duplicates = [smile for smile in unique_list if unique_list.count(smile) > 1]
        if duplicates:
            print("Duplicate SMILES found:", duplicates)
        else:
            pass

Duplicate SMILES found: ['Cc1cc(-c2noc(CCC(=O)N3CCC[C@@H]3C[C@H](O)c3ccc(F)cc3)n2)ccc1F', 'Cc1cc(-c2noc(CCC(=O)N3CCC[C@@H]3C[C@H](O)c3ccc(F)cc3)n2)ccc1F']


In [12]:
dataset_best_sorted = Dataset.load('../datasets/dataset_mqn_shuffled.pkl')
dataset_best_sorted.sort_by_y()

# Ensure every that every SMILES in the original list can be found in the imported dataset.
list = SMILES[10]
for list in SMILES:
    if len(dataset_best_sorted.get_points_from_ids(list).y) != 10:
        print("You done messed up?")

In [13]:
# Test that each newly generated list have 10 items.

for nudged_lists in new_SMILES:
    for unique_list in nudged_lists:
        if len(unique_list) != 10:
            print("You done messed up?")

In [14]:
# Test that every SMILES exists in the dataset.

for i, nudged_lists in enumerate(new_SMILES):
    for j, list in enumerate(nudged_lists):
        if len(dataset.get_points_from_ids(list).y) != 10:
            print("You done messed up?", i, j)

You done messed up? 0 16


In [15]:
dataset.get_points_from_ids(new_SMILES[2][5])

<Dataset X.shape: (10, 42), y.shape: (10,), w.shape: (10,), ids: ['NC(=O)c1cccc(Oc2ccc(NC(=O)[C@H]3C[C@@H]3c3ccc(F)cc3)cn2)c1'
 'C[C@H]([NH3+])[C@@H]1CCCN(c2ccc(-c3ccc(Cl)s3)nn2)C1'
 'CCc1onc(C)c1NC(=O)NCC1CCc2ccccc2CC1'
 'Cc1ccc(F)cc1C(=O)NC(C)(C)c1cn(C)nn1'
 'Cc1cc(F)ccc1-n1ccc(C(=O)N2CCc3ccccc3[C@H]2CO)n1'
 'c1ccc2c(c1)OCC[C@@H]2c1nc(-c2ccc3[nH]cnc3c2)no1'
 'C[C@H](Nc1nc(-c2cccnc2)nc2c1CCC2)c1cccc(S(N)(=O)=O)c1'
 'CCOc1c(Cl)cc(CNc2ccc3nc(C)nn3n2)cc1OC'
 'CNC(=O)[C@@H]1CCC[C@H]1[NH2+][C@H]1[C@@H]2CCO[C@@H]2C12CCCC2'
 'Cc1ccsc1S(=O)(=O)NC(=O)C(F)(F)C1CCOCC1']>

In [16]:
# Sort the dataset by y (lowest y = best)
dataset_best_sorted = Dataset.load('../datasets/dataset_mqn_shuffled.pkl')
dataset_best_sorted.sort_by_y()

# Retrieve the top 100 molecules explicitly
top_100_dataset = dataset_best_sorted.get_top_or_bottom(100)  
top_100_ids = set(top_100_dataset.ids)

# Now, test the fetched molecules (e.g., from new_SMILES)
for i, run in enumerate(new_SMILES):
    # run is a list of 30 lists, each containing 10 SMILES
    for j, mol_list in enumerate(run):
        # Check that the replaced molecules are in top_100
        # Assume you know which indices should have been replaced
        # Or if all should be from top_100 in this scenario
        for smile_id in mol_list:
            if smile_id not in SMILES[j]:  # if it's one of the enriched molecules
                assert smile_id in top_100_ids, f"Enriched ID {smile_id} not in top 100"

In [17]:
from collections import Counter

dataset = Dataset.load('../datasets/dataset_mqn_shuffled.pkl')
dataset.remove_duplicates()
smiles_list = dataset.ids

smiles_counts = Counter(smiles_list)
duplicates = [smile for smile, count in smiles_counts.items() if count > 1]

if duplicates:
    print("Found duplicates:", duplicates)
    print(len(duplicates))
else:
    print("No duplicates found.")

No duplicates found.


In [18]:
# Ensure the enriched molecules are correct by checking they are not in the original list 
# and their indices fall within the valid range (101 to 500).

index_list = []
for nudged_lists in new_SMILES_middle_good:  # Iterate through lists of nudged SMILES (modified structures).
    for unique_list, original_list in zip(nudged_lists, SMILES):  # Pair each nudged list with its corresponding original list.
        for smile in unique_list:  # Iterate through each SMILE in the current nudged list.
            if smile not in original_list:  # Check if the current SMILE is not in the original list.
                mol_index = dataset_best_sorted.get_indices_from_ids([smile])[0]  # Get the index of the molecule in the dataset.
                index_list.append(int(mol_index))
                if mol_index < 101 or mol_index > 500:  # Check if the molecule's index is outside the valid range.
                    print(f"You done messed up?")  # Debug message indicating an invalid molecule index.
print(index_list)

[403, 108, 395, 470, 120, 352, 309, 211, 353, 212, 312, 423, 101, 403, 221, 466, 413, 225, 111, 237, 305, 491, 365, 169, 252, 272, 109, 104, 364, 458, 490, 415, 293, 437, 288, 435, 183, 434, 146, 276, 110, 449, 384, 216, 385, 344, 407, 206, 271, 495, 425, 217, 460, 351, 447, 334, 281, 177, 485, 337, 286, 411, 344, 333, 353, 248, 431, 372, 160, 333, 361, 409, 218, 497, 432, 114, 198, 144, 483, 111, 156, 198, 162, 324, 388, 303, 272, 393, 368, 162, 177, 231, 406, 227, 441, 180, 491, 486, 253, 174, 150, 446, 487, 456, 168, 110, 426, 186, 221, 419, 209, 227, 153, 425, 243, 275, 457, 124, 425, 242, 351, 317, 294, 213, 319, 289, 111, 489, 328, 415, 402, 177, 439, 413, 249, 403, 468, 281, 443, 270, 104, 439, 183, 196, 157, 229, 125, 126, 231, 238, 419, 241, 256, 317, 300, 369, 379, 496, 165, 488, 345, 478, 472, 460, 281, 483, 225, 234, 255, 216, 274, 110, 225, 105, 422, 131, 163, 294, 480, 336, 461, 125, 491, 175, 479, 278, 406, 290, 171, 398, 108, 149, 382, 202, 237, 497, 425, 221, 207, 478,

In [19]:
# Ensure the enriched molecules are correct by checking they are not in the original list 
# and their indices fall within the valid range (100).

index_list = []
for nudged_lists in new_SMILES:  # Iterate through lists of nudged SMILES (modified structures).
    for unique_list, original_list in zip(nudged_lists, SMILES):  # Pair each nudged list with its corresponding original list.
        for smile in unique_list:  # Iterate through each SMILE in the current nudged list.
            if smile not in original_list:  # Check if the current SMILE is not in the original list.
                mol_index = dataset_best_sorted.get_indices_from_ids([smile])[0]  # Get the index of the molecule in the dataset.
                index_list.append(int(mol_index))
                if mol_index > 100:  # Check if the molecule's index is outside the valid range.
                    print(f"You done messed up?")  # Debug message indicating an invalid molecule index.
print(index_list)

[29, 36, 75, 52, 26, 68, 68, 1, 61, 48, 12, 9, 4, 55, 3, 44, 73, 0, 39, 99, 72, 8, 95, 53, 12, 5, 71, 45, 35, 7, 42, 75, 38, 26, 77, 48, 23, 59, 80, 62, 66, 87, 77, 20, 99, 2, 21, 71, 17, 48, 83, 90, 81, 93, 7, 92, 55, 99, 14, 45, 51, 73, 6, 59, 32, 8, 41, 20, 30, 28, 94, 34, 75, 89, 82, 98, 94, 22, 42, 29, 76, 37, 34, 54, 81, 84, 81, 92, 6, 79, 13, 27, 95, 35, 51, 55, 99, 4, 91, 1, 68, 30, 79, 52, 82, 78, 77, 77, 83, 57, 15, 64, 80, 31, 38, 7, 57, 91, 89, 38, 34, 75, 98, 92, 83, 50, 98, 15, 37, 68, 17, 94, 29, 19, 96, 7, 81, 13, 58, 53, 48, 4, 42, 41, 59, 13, 69, 14, 88, 12, 44, 91, 63, 62, 41, 88, 78, 41, 86, 51, 70, 64, 62, 61, 78, 94, 71, 79, 83, 68, 29, 6, 99, 1, 4, 75, 32, 55, 76, 20, 17, 84, 75, 31, 92, 39, 87, 30, 22, 28, 49, 43, 69, 14, 77, 40, 20, 16, 54, 35, 32, 25, 60, 52, 90, 10, 30, 82, 37, 59, 48, 7, 38, 30, 55, 35, 20, 84, 57, 52, 65, 55, 79, 32, 87, 44, 9, 26, 18, 25, 68, 27, 34, 95, 17, 67, 10, 39, 32, 77, 42, 5, 26, 69, 42, 59, 44, 66, 64, 8, 86, 46, 78, 37, 76, 52, 

In [20]:
new_SMILES[4]

[['O=C(Nc1ccc(Nc2ncccn2)cc1)c1cn[nH]c1-c1ccco1',
  'O=C([C@@H]1C[C@@H]2CC[C@H]1O2)N1CCC(c2nc(=O)[nH][nH]2)CC1',
  'COc1ccnc(NC[C@]23C[C@](NC(=O)[C@@H]4C[C@@H]4C)(C2)C(C)(C)O3)n1',
  'Cc1csc(N2CCN(C(=O)c3ccc(C(=O)NC4CC4)cc3)C[C@H]2C)n1',
  'Cn1nc(-c2ccccc2)c2c(NC(=O)[C@H]3C[C@@H]3C(F)F)n[nH]c21',
  'O=C(Nc1cccnc1)c1cccc(S(=O)(=O)N2CCCCC2)c1',
  'CNc1ccc(C(=O)NCc2cccc(N3CCOCC3)c2)cn1',
  'Cc1ccc(C(=O)N2CC[C@H](c3cc(O)[nH]n3)C2)cc1',
  'O=C1CCCc2cc(C(=O)N(CCCO)Cc3cccc(Cl)c3)ccc2N1',
  'O=C(Cc1c[nH]c2ccc(F)cc12)N[C@H]1CCCN(Cc2ccc(F)cc2F)C1=O'],
 ['CNc1cccc(-c2cn(Cc3cc(C(F)(F)F)nn3C)nn2)c1',
  'C[C@@H](NC(=O)c1c[nH]nc1-c1cccnc1)c1ccc(Cl)cc1',
  'Cc1ccccc1CNc1cc(C(N)=O)ccc1Cl',
  'O=C(CC[C@H]1Cc2ccccc2NC1=O)Nc1c(-c2ccccc2)[nH][nH]c1=O',
  'C[C@H](NC(=O)N[C@@H](CO)C[C@@H](O)C(C)(C)C)c1ccc(-c2ccncc2)cc1',
  'O=C(N[C@H]1C[C@@H](Cn2ccnc2)C[C@H]1O)c1ccc2ccccc2c1',
  'Cc1cccc(-c2cccc(C(=O)N3C[C@H](CO)[C@@H](CO)C3)c2)c1',
  'COCCN1C(=O)N[C@H]2CN(C(=O)c3cccc(C#N)c3)CC[C@@H]21',
  'CCc1nocc1CN(C)C(=O

In [27]:
new_SMILES_middle_good[4]

[['O=C(Nc1ccc(Nc2ncccn2)cc1)c1cn[nH]c1-c1ccco1',
  'O=C1CCc2cc(CC(=O)N[C@H]3C[C@H](Cn4cccn4)C[C@@H]3O)ccc2N1',
  'COc1ccnc(NC[C@]23C[C@](NC(=O)[C@@H]4C[C@@H]4C)(C2)C(C)(C)O3)n1',
  'Cc1csc(N2CCN(C(=O)c3ccc(C(=O)NC4CC4)cc3)C[C@H]2C)n1',
  'CN1C(=O)CCc2cc(NC(=O)NC[C@@H](O)c3ccccc3)ccc21',
  'NC(=O)NC[C@@H]1CCCN(C(=O)NCCc2ccccc2C(F)(F)F)C1',
  'Cc1cccc2c(CCN3C(=O)[C@H]4C[C@@H]5CCCC[C@@H]5N4C3=O)c[nH]c12',
  'O=C(Nc1cc(C2CC2)nn1-c1ccccc1)N[C@H](Cn1ccccc1=O)c1ccccc1',
  'Cc1cccc([C@@H]2CCCN2C(=O)NCc2ccc(S(C)(=O)=O)cc2)n1',
  'COc1ccc(CNC(=O)[C@H]2CCCN2C(=O)[C@]23C[C@H]4C[C@H](C[C@H](C4)C2)C3)cn1'],
 ['O=C(NCc1ccc(C(=O)N2CCCCC2)cc1)N1CCn2cncc2C1',
  'COc1cc(-c2noc([C@H]3NCCOc4ccccc43)n2)ccc1C',
  'Cc1ccccc1CNc1cc(C(N)=O)ccc1Cl',
  'COc1ccc(OC)c([C@@H]2CCCN2C(=O)c2ccnc(OC)n2)c1',
  'C=CCN(CC(=O)[O-])S(=O)(=O)c1ccc(OC)c(Cl)c1',
  'CCc1ccc(-c2ccc(C(=O)N(CC(=O)NC)C[C@H]3CCN(S(C)(=O)=O)C3)o2)cc1',
  'COC[C@@H]1CN(C(=O)c2ncoc2C2CC2)CCO1',
  'C[C@H]1C[C@H](CO)CN(C(=O)C(=O)N[C@H]2CC(C)(C)c3ccccc32)C